In [1]:
import pandas as pd
import requests
import warnings
import numpy as np
from sportsdataverse import mbb

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 120)

In [2]:
#pbp = mbb.load_mbb_pbp(range(2011,2023))

In [3]:
#pbp.groupby('participants_0_athlete_id').count().describe()

In [4]:
mbb_sc = mbb.load_mbb_schedule(range(2004,2024))
mbb_sc

100%|██████████| 20/20 [00:08<00:00,  2.43it/s]


,id,uid,date,attendance,time_valid,neutral_site,conference_competition,recent,start_date,notes_type,...,season_type,status_type_alt_detail,groups_id,groups_name,groups_short_name,groups_is_conference,tournament_id,PBP,team_box,player_box
0,244000063,s:40~l:41~e:244000063~c:244000063,2004-04-06T01:21Z,44468,True,False,False,False,2004-04-06T01:21Z,event,...,3,None,None,None,None,None,22.0,False,False,False
1,244000062,s:40~l:41~e:244000062~c:244000062,2004-04-04T01:47Z,44417,True,False,False,False,2004-04-04T01:47Z,event,...,3,None,None,None,None,None,22.0,False,False,False
2,244000031,s:40~l:41~e:244000031~c:244000031,2004-04-03T23:07Z,44417,True,False,False,False,2004-04-03T23:07Z,event,...,3,None,None,None,None,None,22.0,False,False,False
3,240920130,s:40~l:41~e:240920130~c:240920130,2004-04-02T00:00Z,16064,True,False,False,False,2004-04-02T00:00Z,event,...,3,None,None,None,None,None,NaN,False,False,False
4,240900130,s:40~l:41~e:240900130~c:240900130,2004-03-31T02:00Z,12630,True,False,False,False,2004-03-31T02:00Z,event,...,3,None,None,None,None,None,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115019,401489513,s:40~l:41~e:401489513~c:401489513,2022-11-07T21:00Z,3148,True,False,False,False,2022-11-07T21:00Z,,...,2,None,None,None,None,None,NaN,True,True,True
115020,401485025,s:40~l:41~e:401485025~c:401485025,2022-11-07T19:00Z,3380,True,False,False,False,2022-11-07T19:00Z,,...,2,None,None,None,None,None,NaN,True,True,True
115021,401494597,s:40~l:41~e:401494597~c:401494597,2022-11-07T18:00Z,175,True,False,False,False,2022-11-07T18:00Z,,...,2,None,None,None,None,None,NaN,True,True,True
115022,401483265,s:40~l:41~e:401483265~c:401483265,2022-11-07T17:00Z,10284,True,False,False,False,2022-11-07T17:00Z,,...,2,None,None,None,None,None,NaN,True,True,True


In [5]:
mbb_sc.columns

Index(['id', 'uid', 'date', 'attendance', 'time_valid', 'neutral_site',
       'conference_competition', 'recent', 'start_date', 'notes_type',
       'notes_headline', 'type_id', 'type_abbreviation', 'venue_id',
       'venue_full_name', 'venue_address_city', 'venue_address_state',
       'venue_capacity', 'venue_indoor', 'status_clock',
       'status_display_clock', 'status_period', 'status_type_id',
       'status_type_name', 'status_type_state', 'status_type_completed',
       'status_type_description', 'status_type_detail',
       'status_type_short_detail', 'format_regulation_periods', 'home_id',
       'home_uid', 'home_location', 'home_name', 'home_abbreviation',
       'home_display_name', 'home_short_display_name', 'home_color',
       'home_alternate_color', 'home_is_active', 'home_venue_id', 'home_logo',
       'home_conference_id', 'home_score', 'home_winner', 'away_id',
       'away_uid', 'away_location', 'away_name', 'away_abbreviation',
       'away_display_name', 'away

In [6]:
mbb_bs = mbb.load_mbb_team_boxscore(range(2004,2024))

mbb_bs.team_id = mbb_bs.team_id.apply(pd.to_numeric)
mbb_bs = mbb_bs.sort_values(['team_id', 'game_date'])
mbb_bs['largest_lead'] = mbb_bs.largestLead.fillna(0).apply(lambda x: int(x)) + mbb_bs.largest_lead.fillna(0).apply(lambda x: int(x))
mbb_bs = mbb_bs.dropna(axis=1)

mbb_bs['next_gameid'] = mbb_bs.game_id.shift(-1)
mbb_bs['next_teamid'] = mbb_bs.team_id.shift(-1)
mbb_bs['next_home_away'] = mbb_bs.home_away.shift(-1)

mbb_bs[mbb_bs.team_id!=mbb_bs.next_teamid].next_gameid=-1

mbb_bs = mbb_bs.drop('next_teamid', axis=1)

mbb_bs[
        [
            'field_goals_made_field_goals_attempted', 
            'three_point_field_goals_made_three_point_field_goals_attempted', 
            'free_throws_made_free_throws_attempted'
        ]
] = mbb_bs[
        [
            'field_goals_made_field_goals_attempted', 
            'three_point_field_goals_made_three_point_field_goals_attempted', 
            'free_throws_made_free_throws_attempted'
        ]
    ].applymap(lambda x: x.split('-'))

mbb_bs['field_goals_made'] = mbb_bs.field_goals_made_field_goals_attempted.apply(lambda x: x[0])
mbb_bs['field_goals_attempted'] = mbb_bs.field_goals_made_field_goals_attempted.apply(lambda x: x[1])
mbb_bs['three_point_field_goals_made'] = mbb_bs.three_point_field_goals_made_three_point_field_goals_attempted.apply(lambda x: x[0])
mbb_bs['three_point_field_goals_attempted'] = mbb_bs.three_point_field_goals_made_three_point_field_goals_attempted.apply(lambda x: x[1])
mbb_bs['free_throws_made'] = mbb_bs.free_throws_made_free_throws_attempted.apply(lambda x: x[0])
mbb_bs['free_throws_attempted'] = mbb_bs.free_throws_made_free_throws_attempted.apply(lambda x: x[1])

mbb_bs = mbb_bs.drop([
            'field_goals_made_field_goals_attempted', 
            'three_point_field_goals_made_three_point_field_goals_attempted', 
            'free_throws_made_free_throws_attempted'
        ], axis=1)

mbb_bs.home_away = mbb_bs.home_away.apply(lambda x: 1 if x=='HOME' else 0)

100%|██████████| 20/20 [00:09<00:00,  2.21it/s]
/var/folders/9m/cmqzd33j5g99b6zzjsbfrvz80000gn/T/ipykernel_50878/3168056671.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbb_bs[mbb_bs.team_id!=mbb_bs.next_teamid].next_gameid=-1


In [7]:

mbb_bs['next_home_away'] = mbb_bs.home_away.shift(-1)

In [8]:
mbb_bs.columns

Index(['team_id', 'team_uid', 'team_slug', 'team_location', 'team_name',
       'team_abbreviation', 'team_display_name', 'field_goal_pct',
       'three_point_field_goal_pct', 'free_throw_pct', 'total_rebounds',
       'offensive_rebounds', 'defensive_rebounds', 'assists', 'steals',
       'blocks', 'turnovers', 'team_turnovers', 'total_turnovers',
       'technical_fouls', 'total_technical_fouls', 'flagrant_fouls', 'fouls',
       'home_away', 'opponent_id', 'opponent_name', 'opponent_mascot',
       'opponent_abbrev', 'game_id', 'season', 'season_type', 'game_date',
       'largest_lead', 'next_gameid', 'next_home_away', 'field_goals_made',
       'field_goals_attempted', 'three_point_field_goals_made',
       'three_point_field_goals_attempted', 'free_throws_made',
       'free_throws_attempted'],
      dtype='object')

In [9]:
mbb_bs.groupby('team_id').count().describe()

,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,field_goal_pct,three_point_field_goal_pct,free_throw_pct,total_rebounds,...,game_date,largest_lead,next_gameid,next_home_away,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted
count,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,...,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000,1288.000000
mean,161.703416,161.703416,161.703416,161.703416,161.703416,161.703416,161.703416,161.703416,161.703416,161.703416,...,161.703416,161.703416,161.702640,161.702640,161.703416,161.703416,161.703416,161.703416,161.703416,161.703416
std,251.198267,251.198267,251.198267,251.198267,251.198267,251.198267,251.198267,251.198267,251.198267,251.198267,...,251.198267,251.198267,251.198766,251.198766,251.198267,251.198267,251.198267,251.198267,251.198267,251.198267
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
75%,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,...,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000,517.500000
max,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,...,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000,640.000000


In [10]:
mbb_bs

,team_id,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,field_goal_pct,three_point_field_goal_pct,free_throw_pct,...,game_date,largest_lead,next_gameid,next_home_away,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted
669,1,s:40~l:41~t:1,alaska-anchorage-seawolves,Alaska Anchorage,Seawolves,UAA,Alaska Anchorage Seawolves,33.3,40.0,58.3,...,2004-11-25,0,243310231.0,1.0,21,63,6,15,7,12
754,1,s:40~l:41~t:1,alaska-anchorage-seawolves,Alaska Anchorage,Seawolves,UAA,Alaska Anchorage Seawolves,48.1,57.1,68.4,...,2004-11-26,0,253280001.0,0.0,25,52,8,14,13,19
9707,1,s:40~l:41~t:1,alaska-anchorage-seawolves,Alaska Anchorage,Seawolves,UAA,Alaska Anchorage Seawolves,35.9,45.8,50.0,...,2005-11-25,0,253290001.0,0.0,23,64,11,24,3,6
9737,1,s:40~l:41~t:1,alaska-anchorage-seawolves,Alaska Anchorage,Seawolves,UAA,Alaska Anchorage Seawolves,48.7,46.7,75.0,...,2005-11-25,9,253300030.0,1.0,19,39,7,15,27,36
9868,1,s:40~l:41~t:1,alaska-anchorage-seawolves,Alaska Anchorage,Seawolves,UAA,Alaska Anchorage Seawolves,46.7,64.3,71.4,...,2005-11-26,0,263142440.0,1.0,21,45,9,14,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199542,126954,s:40~l:41~t:126954,clinton-college-golden-bears,Clinton College,Golden Bears,CLIN,Clinton College Golden Bears,39.0,28.6,75.0,...,2022-11-10,0,401499091.0,1.0,23,59,4,14,15,20
199244,126955,s:40~l:41~t:126955,cazenovia-college-wildcats,Cazenovia College,Wildcats,CAZE,Cazenovia College Wildcats,29.5,30.4,76.5,...,2022-11-08,0,401499093.0,1.0,18,61,7,23,13,17
201756,126956,s:40~l:41~t:126956,framingham-state-rams,Framingham State,Rams,FRAM,Framingham State Rams,25.0,12.5,62.5,...,2022-11-29,0,401499435.0,1.0,16,64,2,16,10,16
199242,126972,s:40~l:41~t:126972,miami-oh---middletown-thunderhawks,Miami (OH) - Middletown,ThunderHawks,MMT,Miami (OH) - Middletown ThunderHawks,25.4,5.3,95.5,...,2022-11-08,0,401491873.0,1.0,15,59,1,19,21,22


In [12]:
matchups = pd.merge(mbb_bs, mbb_bs, how='inner', on='game_id', suffixes=['_act', '_opp'])
matchups = matchups[matchups.team_id_act!=matchups.team_id_opp]

ind_list = [
    'next_gameid_act',
    'next_home_away_act',

    'game_id',
    'game_date_act',
    'team_uid_act',
    'team_slug_act',
    'team_location_act',
    'team_name_act',
    'team_abbreviation_act',
    'team_display_name_act',
    'opponent_name_act',
    'opponent_mascot_act',
    'opponent_abbrev_act',
    'season_act',
    'season_type_act',
    'team_uid_opp',
    'team_slug_opp',
    'team_location_opp',
    'team_name_opp',
    'team_abbreviation_opp',
    'team_display_name_opp',
    'opponent_id_opp',
    'opponent_name_opp',
    'opponent_mascot_opp',
    'opponent_abbrev_opp',
    'season_opp',
    'season_type_opp',
    'game_date_opp',
    'next_gameid_opp',
    'next_home_away_opp',
]

upcoming = matchups.set_index(ind_list[0:2]).drop(ind_list[2:], axis=1).groupby('team_id_act').rolling(19, min_periods=19, closed='left').mean().dropna().reset_index(drop=False)
upcoming = pd.merge(upcoming[upcoming.next_home_away_act==1], upcoming[upcoming.next_home_away_act==0], how='inner', on='next_gameid_act', suffixes=['_home', '_away'])

upcoming

,team_id_act_home,next_gameid_act,next_home_away_act_home,field_goal_pct_act_home,three_point_field_goal_pct_act_home,free_throw_pct_act_home,total_rebounds_act_home,offensive_rebounds_act_home,defensive_rebounds_act_home,assists_act_home,...,flagrant_fouls_opp_away,fouls_opp_away,home_away_opp_away,largest_lead_opp_away,field_goals_made_opp_away,field_goals_attempted_opp_away,three_point_field_goals_made_opp_away,three_point_field_goals_attempted_opp_away,free_throws_made_opp_away,free_throws_attempted_opp_away
0,1,401089662.0,1.0,40.468421,36.442105,72.405263,30.947368,9.210526,21.052632,15.894737,...,0.0,16.315789,0.578947,9.000000,23.315789,54.473684,5.736842,15.631579,14.263158,21.526316
1,2,250500061.0,1.0,44.194737,33.610526,73.300000,37.315789,11.578947,21.842105,13.842105,...,0.0,19.000000,0.736842,13.526316,24.631579,53.684211,5.578947,16.631579,14.894737,22.684211
2,2,250540096.0,1.0,43.063158,32.805263,73.210526,37.473684,11.736842,21.842105,13.157895,...,0.0,18.684211,0.684211,4.842105,21.263158,54.052632,6.315789,21.210526,10.421053,15.052632
3,2,250610333.0,1.0,41.389474,30.778947,72.821053,38.263158,12.157895,21.947368,12.526316,...,0.0,18.631579,0.421053,9.210526,26.473684,59.105263,7.105263,20.105263,16.894737,22.578947
4,2,250690238.0,1.0,40.131579,30.310526,74.694737,37.315789,11.842105,21.473684,12.105263,...,0.0,18.000000,0.473684,12.789474,26.894737,56.947368,5.421053,15.789474,14.842105,20.578947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91413,112358,320052529.0,1.0,44.247368,32.810526,71.463158,38.631579,11.263158,27.157895,15.473684,...,0.0,17.263158,0.421053,13.421053,27.105263,58.578947,7.684211,21.052632,11.631579,17.368421
91414,112358,400841201.0,1.0,44.831579,35.031579,74.421053,39.736842,10.684211,28.789474,14.947368,...,0.0,19.526316,0.526316,4.052632,24.789474,58.578947,5.894737,18.210526,15.894737,22.263158
91415,112358,400919153.0,1.0,44.257895,34.473684,72.978947,40.052632,10.578947,29.210526,14.578947,...,0.0,15.789474,0.105263,17.894737,25.421053,56.473684,8.052632,22.421053,15.947368,23.368421
91416,112358,263322599.0,1.0,44.821053,35.342105,72.368421,39.789474,10.210526,29.315789,14.052632,...,0.0,19.105263,0.578947,6.105263,22.210526,59.315789,6.947368,22.789474,12.105263,18.263158


In [13]:
upcoming.columns.to_list()

['team_id_act_home',
 'next_gameid_act',
 'next_home_away_act_home',
 'field_goal_pct_act_home',
 'three_point_field_goal_pct_act_home',
 'free_throw_pct_act_home',
 'total_rebounds_act_home',
 'offensive_rebounds_act_home',
 'defensive_rebounds_act_home',
 'assists_act_home',
 'steals_act_home',
 'blocks_act_home',
 'turnovers_act_home',
 'team_turnovers_act_home',
 'total_turnovers_act_home',
 'technical_fouls_act_home',
 'total_technical_fouls_act_home',
 'flagrant_fouls_act_home',
 'fouls_act_home',
 'home_away_act_home',
 'opponent_id_act_home',
 'largest_lead_act_home',
 'field_goals_made_act_home',
 'field_goals_attempted_act_home',
 'three_point_field_goals_made_act_home',
 'three_point_field_goals_attempted_act_home',
 'free_throws_made_act_home',
 'free_throws_attempted_act_home',
 'team_id_opp_home',
 'field_goal_pct_opp_home',
 'three_point_field_goal_pct_opp_home',
 'free_throw_pct_opp_home',
 'total_rebounds_opp_home',
 'offensive_rebounds_opp_home',
 'defensive_rebounds_

In [14]:
temp = pd.merge(upcoming, matchups[matchups['home_away_act']==1], left_on='next_gameid_act', right_on='game_id', how='inner')

temp['final_spread'] = ((temp.free_throws_made_act.astype(int)+temp.field_goals_made_act.astype(int)*2+temp.three_point_field_goals_made_act.astype(int)*3) 
                      - (temp.free_throws_made_opp.astype(int)+temp.field_goals_made_opp.astype(int)*2+temp.three_point_field_goals_made_opp.astype(int)*3))

#upcoming.set_index(['team_id_act_home', 'team_id_act_away', 'next_gameid_act']).drop(['next_home_away_act_home', 'opponent_id_act_home'], axis=1)

In [20]:
temp.final_spread

0       -36
1       -22
2         4
3       -51
4        -8
         ..
91413   -12
91414     0
91415   -33
91416   -24
91417     8
Name: final_spread, Length: 91418, dtype: int64

In [16]:
(mbb_sc.home_score.astype(int)-mbb_sc.away_score.astype(int)).mean()

6.81141327027403

In [21]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

X = upcoming
Y = temp.final_spread

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

clf = XGBRegressor()

clf.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [25]:
from sklearn.metrics import mean_squared_error
import math

math.sqrt(mean_squared_error(Y_test, clf.predict(X_test)))

17.808756842539196